In [2]:
# %pip install pyspark==3.4.1

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# from pyspark.sql.functions import array, col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import datetime

In [4]:
import os
cwd = os.getcwd()

In [5]:
spark = SparkSession.builder \
                    .master("spark://127.0.0.1:7077") \
                    .appName('TradeCropsLivestock') \
                    .getOrCreate()
# spark = SparkSession.builder \
#                     .master("local[1]") \
#                     .appName('TradeCropsLivestock') \
#                     .getOrCreate()

In [6]:
tcl_df = spark.read.option("header", True)\
    .csv(f"{cwd}/dataset/Trade_CropsLivestock_E_All_Data/Trade_CropsLivestock_E_All_Data.csv")
# tcl_df.printSchema()

In [7]:
col_name = []
for col in tcl_df.dtypes:
    col_name.append(col[0])
# col_name

In [8]:
year_list = []
for x in col_name:
    if 'Y' in x and 'F' not in x:
        year_list.append(int(x.replace('Y', '')))
# year_list

In [9]:
ElementTrade_df = tcl_df.select(
    tcl_df['Element Code'].cast('int').alias('id'),
    tcl_df['Element'].cast('string').alias('element'),
    tcl_df['Unit'].cast('string').alias('unit'),
).orderBy('id')
ElementTrade_df = ElementTrade_df.dropDuplicates(["id"])
ElementTrade_df.show()

+----+---------------+---------+
|  id|        element|     unit|
+----+---------------+---------+
|5607|Import Quantity|       No|
|5608|Import Quantity|     Head|
|5609|Import Quantity|1000 Head|
|5610|Import Quantity|   tonnes|
|5622|   Import Value| 1000 US$|
|5907|Export Quantity|       No|
|5908|Export Quantity|     Head|
|5909|Export Quantity|1000 Head|
|5910|Export Quantity|   tonnes|
|5922|   Export Value| 1000 US$|
+----+---------------+---------+



In [10]:
for year in year_list:
    tcl_df = tcl_df.withColumn(f"NewColumn_{year}", F.array(f"Y{year}", f"Y{year}F", F.lit(year)))
    tcl_df = tcl_df.drop(f"Y{year}", f"Y{year}F")
    
tcl_df = tcl_df.withColumn(f"merge_column", F.array(*[f"NewColumn_{year}" for year in year_list]))
transformed_date = datetime.datetime.now()
for year in year_list:
    tcl_df = tcl_df.drop(f"NewColumn_{year}")
tcl_df = tcl_df.select(
    tcl_df['Area Code'].cast('int').alias('country_id'),
    tcl_df['Item Code'].cast('int').alias('item_prod_id'),
    tcl_df['Element Code'].cast('int').alias('element_trade_id'),
    F.explode('merge_column').alias('year_col')
)
tcl_df = tcl_df.withColumn('value', F.lit(tcl_df.year_col[0])) \
                .withColumn('flag', F.lit(tcl_df.year_col[1])) \
                .withColumn('year', F.lit(tcl_df.year_col[2])) \
                .withColumn('transformed_date', F.lit(transformed_date))
tcl_df = tcl_df.drop('year_col')
tcl_df.show()

+----------+------------+----------------+-----+----+----+--------------------+
|country_id|item_prod_id|element_trade_id|value|flag|year|    transformed_date|
+----------+------------+----------------+-----+----+----+--------------------+
|         2|         221|            5610| null|null|1961|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1962|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1963|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1964|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1965|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1966|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1967|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1968|2023-07-26 23:00:...|
|         2|         221|            5610| null|null|1969|2023-07-26 23:00:...|
|         2|         221|            561

In [11]:
ElementTrade_df.write\
    .mode("overwrite") \
    .parquet("hdfs://127.0.0.1:9000/FAOSTAT_prj/DataWarehouse/Element_Trade")

In [12]:
tcl_df.write\
    .mode("overwrite") \
    .parquet("hdfs://127.0.0.1:9000/FAOSTAT_prj/DataWarehouse/Trade_Crops_Livestock")

In [13]:
spark.stop()